In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging
logging.langsmith("CH01-Basic")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = PromptTemplate.from_template("{topic}에 대해 3문장으로 설명해줘.")
chain = prompt | model | StrOutputParser()

입력 변수 {topic}에 ChatGPT와 Instagram을 넣으려고 한다.
batch() 함수로 각각 호출해서 답변을 얻는 대신 배치 단위로 묶어 한번에 처리

In [ ]:
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

필요하다면 배치 처리 결과를 answer에 저장하고, 인덱스 번호를 붙여 답변을 따로 출력


In [ ]:
answer = chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])
answer[0]

이번에는 배치 단위로 5개의 질문을 처리
batch() 함수의 동작 방식을 설명하는 config 매개변수 추가
max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정

In [ ]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

flush=True는 출력 버퍼를 즉시 비워 메시지가 바로 출력되게 함

In [ ]:
async for token in chain.astream({"topic": "YouTube"}):
  print(token, end="", flush=True)

`ainvoke()` 함수는 특정 토픽에 대한 처리를 비동기적으로 요청하는데 사용

In [14]:
my_process = chain.ainvoke({"topic": "NVDA"})

# 비동기 작업이 완료될때까지 프로그램이 진행되지 않도록 await 키워드 사용
await my_process

'NVDA는 미국의 기업으로서 그래픽카드와 컴퓨팅 시스템을 제조하는 선도적인 기업이다. 주로 게임용 그래픽카드 및 딥러닝 및 자율주행 분야에서의 기술력으로 유명하며, 시장에서 높은 점유율을 보유하고 있다. 높은 수익률과 성장을 보여주는 기업으로 투자자들의 관심을 끌고 있다.'

topic과 관련된 여러 작업들을 동시에 비동기 방식으로 처리

In [15]:
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)
await my_abatch_process

['YouTube는 동영상을 업로드, 시청 및 공유할 수 있는 온라인 비디오 플랫폼이다. 각종 콘텐츠뿐만 아니라 뉴스, 뮤직비디오, 강의, 블로그 등 다양한 영상 콘텐츠를 제공한다. 전 세계적으로 많은 이용자들이 이용하고 있으며, 크리에이터들은 수익을 창출할 수 있는 기회를 제공받는다.',
 'Instagram은 사진이나 짧은 동영상을 공유할 수 있는 소셜 미디어 플랫폼이다. 사용자들은 다른 사람들의 포스트에 좋아요를 누르거나 댓글을 남길 수 있으며 팔로우하여 최신 소식을 받아볼 수 있다. 또한 이용자들은 해시태그를 통해 관심사나 주제에 맞는 게시물을 찾을 수 있다.',
 'Facebook은 세계적으로 가장 인기 있는 소셜 네트워킹 서비스 중 하나로, 사진과 동영상을 공유하고 다른 사람들과 소통할 수 있는 플랫폼이다. 사용자들은 친구, 가족 또는 동료와 연결되어 콘텐츠를 공유하며 소셜 네트워크를 형성할 수 있다. 또한 광고 및 마케팅 등 다양한 비즈니스 활동을 할 수 있는 기회도 제공한다.']

RunnableParallel 클래스를 활용해 두가지 작업을 동시에 처리

In [16]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

두 체인을 병렬로 수행하기 위해 RunnableParallel 클래스 사용
두 체인의 실행 결과를 구분해서 받기 위해 각각의 체인에 고유한 키를 할당
키를 설정하면 실행 결과를 받을 때 각 키를 통해 원하는 정보 쉽게 접근 가능

In [18]:
# 앞에서 만든 두 개 체인을 동시에 생성하는 병렬 실행 체인 생성
combined = RunnableParallel(capital=chain1, area=chain2)

In [20]:
chain1.invoke({'country': '대한민국'})

'대한민국의 수도는 서울이다.'

In [22]:
chain2.invoke({'country': '미국'})

'미국의 총 면적은 대략 9,833,520km²입니다.'

In [23]:
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 면적은 약 100,363km² 입니다.'}

배치 처리를 통해 여러 데이터를 한번에 입력하고, 그에 대한 답변도 한꺼번에 받을 수 있다

In [24]:
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울이에요.', '미국의 수도는 워싱턴 D.C. 입니다.']

In [25]:
chain2.invoke([{"country": "대한민국"}, {"country": "미국"}])


'이 두 국가의 면적은 다음과 같습니다.\n\n- 대한민국: 약 100,363㎢\n- 미국: 약 9,826,675㎢\n\n두 국가의 면적을 합산하면 약 9,926,038㎢가 됩니다.'

combined.batch() 함수로 주어진 데이터 배치로 처리

In [30]:
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이야.', 'area': '대한민국의 면적은 약 100,363제곱킬로미터입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '미국의 총 면적은 약 9,833 만 제곱 킬로미터입니다.'}]